In [2]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np
import pandas as pd
import os
import shutil

#import glob2

#from numba import jit

colors = [
'black','blue','red','green',
'cyan',  
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']

Autosaving every 180 seconds


In [13]:
# function that upsamples stack of waveforms up to some factor
def upsample_resample(wf, upsample_factor):
    
    # grab the max channel template from each of the first tempaltes
    wf_local = []
    for k in range(wf.shape[0]):
        wf_local.append(wf[k,:,max_chans[k]])

    wf = np.array(wf_local)

    wf = wf.T
    waveform_len, n_spikes = wf.shape
    traces = np.zeros((n_spikes, (waveform_len-1)*upsample_factor+1),'float32')
    for j in range(wf.shape[1]):
        traces[j] = signal.resample(wf[:,j],(waveform_len-1)*upsample_factor+1)
    return traces, wf.T

In [14]:
# load template file
temps = np.load('/media/cat/4TBSSD/data/retina/2009/templates_0sec.npy')
print (temps.shape)

# grab the first 10 templates
temps1 = temps[:10]
print (temps1.shape)

# compute max channels for first 10 templates
max_chans = temps1.ptp(1).argmax(1)

(2413, 101, 512)
(10, 101, 512)


In [20]:
# upsample shift
from scipy import signal

# upsample max channel templates for first 10 templates by a factor
upsample_factor = 5
wf_upsampled, wf_local = upsample_resample(temps1, upsample_factor)
print (wf_upsampled.shape)

# plot upsampled templates on top of original templates
ax = plt.subplot(1,2,1)

# original templates
t = np.arange(wf_local.shape[1])
plt.plot(t, wf_local.T,c='black')

# upsampled tempaltes
tt = np.arange(wf_upsampled.shape[1])/upsample_factor+0.5
plt.plot(tt, wf_upsampled.T)
plt.title("First 10 templates max channel \noriginal sample rate (black) vs. upsampled (colors)")

# plot a single example of downsampled shifted templates
ax = plt.subplot(1,2,2)
shifts = np.arange(0,upsample_factor,1)

wf_downsampled = []
unit = 0
shifts = np.arange(0,5,1)
for shift in shifts:
    print (wf_upsampled[unit,shift::upsample_factor].shape)
    wf_downsampled.append(wf_upsampled[unit,shift::upsample_factor])
    
wf_downsampled=np.array(wf_downsampled)
print (wf_downsampled.shape)
    
for k in range(shifts.shape[0]):
    plt.plot(t[:wf_downsampled[k].shape[0]], wf_downsampled[k])
    plt.plot(t, wf_local[unit],linewidth=2, c='black')

plt.title("First template original sample rate (black) \nvs. upsampled+shifted+downsampled (colors)")

plt.show()

(10, 501)
(101,)
(100,)
(100,)
(100,)
(100,)
(5,)
